# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [14]:
# Na Aula 1 aprendemos a utilizar o pyautogui e o fizemos na web. Agora vamos usar o Selenium, pois ele possui algumas vantagens:
# O Selenium, diferente do pyautogui, controla o navegador de forma inteligente. Ou seja, se um item muda de posição no seu navegador, o selenium continua funcionando.
# O Selenium pode rodar em segundo plano enquanto você utiliza seu computador diferente do pyautogui que precisa "monopolizá-lo"

from selenium import webdriver # controla o navegador
from selenium.webdriver.common.keys import Keys # controla o teclado
from selenium.webdriver.common.by import By # localizar os itens no navegador

# Passo 1 - Criar o navegador
navegador = webdriver.Chrome()

# Passo 2 - Pesquisar a cotação do Dólar
navegador.get("https://www.google.com.br/") # entra no google
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar", Keys.ENTER) # escreve na barra de pesquisa e tecla enter
dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") # pega o atributo com o valor
print(dolar)

# Passo 3 - Fazendo o mesmo para o Euro  
navegador.get("https://www.google.com.br/") 
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro", Keys.ENTER)
euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(euro)

# Passo 4 - Para o Ouro vai ser diferente
navegador.get("https://www.melhorcambio.com/ouro-hoje") # site confiável para cotação do ouro
ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
ouro = ouro.replace(",", ".") # O python é padrão americano, portanto, para usarmos o valor mais tarde sem ter problemas, precisamos substituir a vírgula pelo ponto
print(ouro)

# Passo 5 - Fechar o navegador
navegador.quit()

5.011299999999999
5.511177175
324.63


In [15]:
import pandas as pd
from IPython.display import display

# Passo 6 - Atualizar as cotações na Base de Dados
tabela = pd.read_excel("Produtos.xlsx")
display(tabela) # exibe tabela desatualizada 

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [16]:
# Passo 7 - Atualizar as cotações na tabela
# formato tabela.loc["linhas", "colunas"] (função localiza)
tabela.loc[tabela["Moeda"]=="Dólar", "Cotação"] = float(dolar) 
tabela.loc[tabela["Moeda"]=="Euro", "Cotação"] = float(euro) 
tabela.loc[tabela["Moeda"]=="Ouro", "Cotação"] = float(ouro) 

# Passo 8 - Atualizar preço de compra e preço de venda com base na cotação atualizada
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"] # preço de compra = preço original * cotação
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"] # preço de venda = preço de compra * margem 

display(tabela) # exibe tabela atualizada 

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.011300,5011.249887,1.40,7015.749842
1,Carro Renault,4500.00,Euro,5.511177,24800.297288,2.00,49600.594575
2,Notebook Dell,899.99,Dólar,5.011300,4510.119887,1.70,7667.203808
3,IPhone,799.00,Dólar,5.011300,4004.028700,1.70,6806.848790
4,Carro Fiat,3000.00,Euro,5.511177,16533.531525,1.90,31413.709898
5,Celular Xiaomi,480.48,Dólar,5.011300,2407.829424,2.00,4815.658848
6,Joia 20g,20.00,Ouro,324.630000,6492.600000,1.15,7466.490000


In [17]:
# Passo 9 - Exportar Base de Dados atualizada de volta para o Excel
tabela.to_excel("Produtos.xlsx", index=False) # index=False para exportar sem a enumeração de colunas